In [178]:
from PIL import Image,ImageEnhance,ImageFilter
import torch
from torchvision import transforms,models
import torch.nn as nn

In [179]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [180]:
mean=[0.485,0.456,0.406]
std=[0.229,0.224,0.225]

In [181]:
pre=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean,std)
])

In [182]:
def depreprocess(tensor):
  t=tensor.clone()
  for c in range(3):
    t[c]=t[c]*std[c]+mean[c]
  return torch.clamp(t, 0.0, 1)

In [183]:
def tensor_to_pil(t):
  t = t.squeeze(0).cpu().clone()
  t = depreprocess(t)
  return transforms.ToPILImage()(t)

In [184]:
pth='/content/Screenshot 2025-10-15 160131.png'
img=Image.open(pth).convert('RGB')
timg=pre(img).unsqueeze(0).to(device)

In [185]:
timg.shape

torch.Size([1, 3, 89, 93])

In [186]:
model=models.vgg16(pretrained=True).to(device).eval()

/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [187]:
for param in model.parameters():
  param.requires_grad=False

In [198]:
lay=[8,15,22,29]
wei=[0.5,1,2,4]

In [199]:
class Extract(nn.Module):
  def __init__(self,feat,lay):
    super().__init__()
    self.feat=feat
    self.lay=set(lay)
  def forward(self,x):
    acti=[]
    for i,lays in enumerate(self.feat):
      x=lays(x)
      if i in self.lay:
        acti.append(x)
    return acti

In [200]:
ext=Extract(model.features,lay).to(device)

In [201]:
def losss(acti,wei=None):
  if wei is None:
    wei=[1.0]*len(acti)
  loss=0.0
  for act,w in zip(acti,wei):
    loss+=w*act.pow(2).mean()
  return loss

In [202]:
import torch.optim as optim
import numpy as np

In [203]:
def asc(timg,lr=0.09,jitter=12):
  ox,oy=np.random.randint(-jitter,jitter+1,2)
  timg.data=torch.roll(timg.data,shifts=(ox,oy),dims=(2,3))
  activ=ext(timg)
  loss=-losss(activ,wei)
  timg.grad=None
  loss.backward()
  grad=timg.grad.data
  timg.data+=lr*grad/(grad.std()+1e-8)
  timg.data=torch.roll(timg.data,shifts=(-ox,-oy),dims=(2,3))
  timg.data=torch.clamp(timg.data,-2.2,2.6)
  return loss.item()

In [204]:
def oct(imgpil,octa=4,octscale=1.5,itr=80):
  octimg=[imgpil]
  for i in range(1,octa):
    nsize=(int(imgpil.width/(octscale**i)),int(imgpil.height/(octscale**i)))
    if nsize[0]<64 or nsize[1]<64:
      break
    octimg.append(imgpil.resize(nsize,Image.LANCZOS))
  octimg=octimg[::-1]
  det=None
  for octas in octimg:
    if det is None:
      cur=octas
    else:
      det=det.resize(octas.size,Image.LANCZOS)
      cur=Image.blend(octas,det,alpha=0.5)
    imgten=pre(cur).unsqueeze(0).to(device)
    imgten.requires_grad_(True)
    for i in range(itr):
      asc(imgten)
    det=tensor_to_pil(imgten)
  return det

In [205]:
def make_creepier(img):
    # Sharpen FIRST before other effects
    img = img.filter(ImageFilter.UnsharpMask(radius=1.5, percent=120, threshold=3))

    # Increase contrast for harsh, unsettling look
    enhancer = ImageEnhance.Contrast(img)
    img = enhancer.enhance(1.3)

    # Slightly desaturate for uncanny valley effect
    enhancer = ImageEnhance.Color(img)
    img = enhancer.enhance(0.8)

    # Sharpen again after color adjustments
    img = img.filter(ImageFilter.SHARPEN)

    # Darken slightly for ominous mood
    enhancer = ImageEnhance.Brightness(img)
    img = enhancer.enhance(0.92)

    return img

In [206]:
final = make_creepier(oct(
    imgpil=img,octa=4,octscale=1.4,itr=80
))

In [207]:
final.save("deepdream_output.png")